In [2]:
import sys
import os
import pandas as pd
import warnings
import json
from pydantic import BaseModel

# Add the src folder to the path
sys.path.append(os.path.abspath('..'))
warnings.filterwarnings("ignore", category=FutureWarning)

# Import the classes
from src.data_tool.yahoo_finance import yahoo_finance
from src.data_tool.get_data_fd import get_prices, get_financial_metrics, search_line_items, get_insider_trades, get_company_news
from src.data_tool.ploygon import polygon
from src.data_tool.financial_datasets import FinancialDatasets
from src.data_tool.connect_db import ConnectDB

from src.llm.api_call import call_llm
from langchain_core.prompts import ChatPromptTemplate

db =ConnectDB()
engine = db.get_engine()

FINANCIAL_DATASETS_API_KEY is set
DB_USER and DB_PASSWORD are set


In [3]:
fd = FinancialDatasets()

In [4]:
ticker = 'NVDA'
start_date = '2025-03-01'
end_date = '2025-03-31'

In [5]:
news = fd.get_news(ticker, start_date, end_date)

In [6]:
news

[CompanyNews(ticker='NVDA', title='Nvidia Stock Keeps Heading Lower. Is It Time to Buy?', author='Jeremy Bowman', source='The Motley Fool', date='2025-03-31T19:37:50Z', url='https://www.fool.com/investing/2025/03/31/nvidia-stock-keeps-heading-lower-is-it-time-to-buy/?source=iedfolrf0000001', sentiment='neutral'),
 CompanyNews(ticker='NVDA', title='9 Amazing Tech Stocks Smart Investors Should Not Ignore', author='Jose Najarro', source='The Motley Fool', date='2025-03-31T17:34:54Z', url='https://www.fool.com/investing/2025/03/31/9-amazing-tech-stocks-smart-investors-should-not-i/?source=iedfolrf0000001', sentiment='neutral'),
 CompanyNews(ticker='NVDA', title="CoreWeave IPO Performance Likely Due To Company, Not AI Sector: Analyst Says 'Investor Sentiment Remains Mixed'", author='Chris Katje', source='Benzinga', date='2025-03-31T16:12:32Z', url='https://www.benzinga.com/analyst-ratings/analyst-color/25/03/44566197/coreweave-ipo-performance-likely-due-to-company-not-ai-sector-analyst-says

In [7]:
def prepare_news_for_llm(news_list):
    """
    Prepare company news data for an LLM by removing URLs and converting to a structured format.
    
    Args:
        news_list: List of CompanyNews objects
        
    Returns:
        List of dictionaries containing cleaned news data
    """
    processed_news = []
    
    for news_item in news_list:
        # Create a dictionary with all fields except URL
        news_dict = {
            "ticker": news_item.ticker,
            "title": news_item.title,
            "source": news_item.source,
            "date": news_item.date,
        }
        
        processed_news.append(news_dict)
    
    return processed_news

In [8]:
LLM_news = prepare_news_for_llm(news)
print(LLM_news)

[{'ticker': 'NVDA', 'title': 'Nvidia Stock Keeps Heading Lower. Is It Time to Buy?', 'source': 'The Motley Fool', 'date': '2025-03-31T19:37:50Z'}, {'ticker': 'NVDA', 'title': '9 Amazing Tech Stocks Smart Investors Should Not Ignore', 'source': 'The Motley Fool', 'date': '2025-03-31T17:34:54Z'}, {'ticker': 'NVDA', 'title': "CoreWeave IPO Performance Likely Due To Company, Not AI Sector: Analyst Says 'Investor Sentiment Remains Mixed'", 'source': 'Benzinga', 'date': '2025-03-31T16:12:32Z'}, {'ticker': 'NVDA', 'title': 'Why Nvidia Stock Is Sinking Today', 'source': 'The Motley Fool', 'date': '2025-03-31T16:06:54Z'}, {'ticker': 'NVDA', 'title': 'Nasdaq Sell-Off: After Losing Nearly $800 Billion in Market Cap, Is Nvidia Stock a Buy Anymore? History Offers a Clear Indicator of What Could Happen Next.', 'source': 'The Motley Fool', 'date': '2025-03-31T13:30:00Z'}, {'ticker': 'NVDA', 'title': "Prediction: Buying This Artificial Intelligence (AI) Stock Will Set You Up for Life (And It's Not Pal

In [9]:
print(type(LLM_news))

<class 'list'>


In [18]:
template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide a summary of the news and the sentiment of the news.
                """,
            ),
            (
                "human",
                """Based on the following news, create the investment signal:

                News Data for {ticker}:
                {llm_news}

                Return the trading sentiment in the following JSON format exactly:
                {{
                  "sentiment": "bullish" | "bearish" | "neutral",
                  "confidence": float between 0 and 100,
                  "reasoning": "string"
                }}
                """,
            ),
        ]
    )

prompt = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker})

In [19]:
print(prompt)

messages=[SystemMessage(content='You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide a summary of the news and the sentiment of the news.\n                ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Based on the following news, create the investment signal:\n\n                News Data for NVDA:\n                [\n  {\n    "ticker": "NVDA",\n    "title": "Nvidia Stock Keeps Heading Lower. Is It Time to Buy?",\n    "source": "The Motley Fool",\n    "date": "2025-03-31T19:37:50Z"\n  },\n  {\n    "ticker": "NVDA",\n    "title": "9 Amazing Tech Stocks Smart Investors Should Not Ignore",\n    "source": "The Motley Fool",\n    "date": "2025-03-31T17:34:54Z"\n  },\n  {\n    "ticker": "NVDA",\n    "title": "CoreWeave IPO Performance Likely Due To Company, Not AI Sector: Analyst Says \'Investor Sentiment Remains Mixed\'",\n    "source": "Benzinga",\n    "date": "2025-03-31T16:12:32Z"\n  },\n  {\n   

In [20]:
class sentiment(BaseModel):
    sentiment: str
    confidence: float
    reasoning: str


In [21]:
model_name = "gemini-2.0-flash"
model_provider = "Gemini"
pydantic_model = sentiment
max_retries = 3

In [22]:
sentiment_result = call_llm(prompt, model_name, model_provider, pydantic_model, max_retries=max_retries)
print(sentiment_result)

sentiment='neutral' confidence=65.0 reasoning="The news surrounding Nvidia (NVDA) presents a mixed sentiment. While some articles suggest it's a good time to buy due to recent price drops and long-term AI growth potential, others highlight concerns such as increased export restrictions to China, billionaire investors reducing their NVDA holdings, and competition from other companies. The frequent mentions of 'buy the dip' strategies suggest recent price weakness. While Nvidia's AI event and collaborations with major companies like GM are positive, the overall picture is one of uncertainty and volatility, leading to a neutral sentiment."


In [15]:
sentiment_result

sentiment(sentiment='Error in analysis, using default', confidence=0.0, reasoning='Error in analysis, using default')

In [16]:
print(type(sentiment_result))

<class '__main__.sentiment'>
